In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine('postgres:///music')
conn = engine.connect()
engine.table_names()

['reviews', 'artists', 'genres', 'labels', 'years', 'content']

In [3]:
def postgres_to_df(table_name):
    result = conn.execute(f'SELECT * FROM {table_name}')
    columns = [x for x in result.keys()]
    df = pd.DataFrame(result.fetchall(), columns=columns)
    return df

In [4]:
reviews = postgres_to_df('reviews')
len(reviews)

18393

In [5]:
query = """
SELECT
    reviews.score,
    reviews.reviewid,
    genres.genre
FROM
    reviews
FULL OUTER JOIN genres ON reviews.reviewid = genres.reviewid
ORDER BY reviews.reviewid
"""
result = conn.execute(query)
df = pd.DataFrame(result.fetchall(), columns=result.keys())

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22690 entries, 0 to 22689
Data columns (total 3 columns):
score       22690 non-null float64
reviewid    22690 non-null int64
genre       20319 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 531.9+ KB


In [7]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20319 entries, 0 to 22689
Data columns (total 3 columns):
score       20319 non-null float64
reviewid    20319 non-null int64
genre       20319 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 635.0+ KB


In [8]:
metal = df[df['genre'] == 'metal']
jazz = df[df['genre'] == 'jazz']
len(df), len(metal), len(jazz)

(20319, 862, 435)

In [9]:
metal['score'].mean(), jazz['score'].mean()

(6.949999999999989, 7.303908045977008)

In [10]:
for genre in df['genre'].unique():
    score = df[df['genre'] == genre]['score'].mean()
    print(f'Genre: {genre}')
    print(f'Score: {score}\n')

Genre: rock
Score: 6.942011019283791

Genre: electronic
Score: 6.92070211667529

Genre: jazz
Score: 7.303908045977008

Genre: metal
Score: 6.949999999999989

Genre: rap
Score: 6.900513149454774

Genre: experimental
Score: 7.336694214876028

Genre: pop/r&b
Score: 6.883798882681565

Genre: global
Score: 7.429680365296805

Genre: folk/country
Score: 7.202189781021892

